In [1]:
from MySQLdb import connect
import pandas as pd

/var/folders/fb/xl0hnskx527bfqkv38l3sn7h0000gn/T/ipykernel_32633/3687392072.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Connect to your MySQLdb
#conn = connect(host='localhost', user='root')
conn = connect(host='127.0.0.1', user='root', password = 'root', port = 8890)

In [3]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [4]:
# Return data from the Customers table:
cur.execute('''SHOW DATABASES;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,Database
0,information_schema
1,funkacernette
2,mysql
3,performance_schema
4,sqlw3s
5,sys
6,testDB


### SQL FOREIGN KEY Constraint
The FOREIGN KEY constraint is used to prevent actions that would destroy links between tables.

A FOREIGN KEY is a field (or collection of fields) in one table, that refers to the PRIMARY KEY in another table.

The table with the foreign key is called the child table, and the table with the primary key is called the referenced or parent table.

Look at the following two tables:

Persons Table

In [5]:
pd.read_sql_query('''SELECT * FROM testDB.Persons;''', conn)

/var/folders/fb/xl0hnskx527bfqkv38l3sn7h0000gn/T/ipykernel_32633/1395910305.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''SELECT * FROM testDB.Persons;''', conn)


,ID,LastName,FirstName,Age


In [9]:
cur.execute('''INSERT INTO testDB.Persons VALUES (1, 'Cooper', 'Álice', null); ''')
cur.execute('''INSERT INTO testDB.Persons VALUES (2, 'Cooper', 'Álice', null); ''')
cur.execute('''INSERT INTO testDB.Persons VALUES (90, 'Cooper', 'Álice', null); ''')

1

In [10]:
pd.read_sql_query('''SELECT * FROM testDB.Persons;''', conn)

/var/folders/fb/xl0hnskx527bfqkv38l3sn7h0000gn/T/ipykernel_32633/1395910305.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''SELECT * FROM testDB.Persons;''', conn)


,ID,LastName,FirstName,Age
0,1,Cooper,Álice,None
1,2,Cooper,Álice,None
2,90,Cooper,Álice,None


Orders Table

In [6]:
pd.read_sql_query('''SELECT * FROM testDB.Orders;''', conn)

/var/folders/fb/xl0hnskx527bfqkv38l3sn7h0000gn/T/ipykernel_32633/1483630969.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''SELECT * FROM testDB.Orders;''', conn)


,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2
...,...,...,...,...,...
191,10439,51,6,1997-02-07,3
192,10440,71,4,1997-02-10,2
193,10441,55,3,1997-02-10,2
194,10442,20,3,1997-02-11,2


Notice that the "PersonID" column in the "Orders" table points to the "PersonID" column in the "Persons" table.

The "PersonID" column in the "Persons" table is the PRIMARY KEY in the "Persons" table.

The "PersonID" column in the "Orders" table is a FOREIGN KEY in the "Orders" table.

The FOREIGN KEY constraint prevents invalid data from being inserted into the foreign key column, because it has to be one of the values contained in the parent table.E TABLE

In [21]:
cur.execute('''DROP TABLE testDB.ORDERS;''')

0

In [22]:
# The following SQL creates a PRIMARY KEY on the "ID" column when the "Persons" table is created:
cur.execute('''CREATE TABLE testDB.Orders (
    OrderID int NOT NULL,
    OrderNumber int NOT NULL,
    PersonID int,
    PRIMARY KEY (OrderID),
    FOREIGN KEY (PersonID) REFERENCES Persons(ID)
);''')

0

SQL Server / Oracle / MS Access:

CREATE TABLE Orders (
    OrderID int NOT NULL PRIMARY KEY,
    OrderNumber int NOT NULL,
    PersonID int FOREIGN KEY REFERENCES Persons(PersonID)
);

To allow naming of a FOREIGN KEY constraint, and for defining a FOREIGN KEY constraint on multiple columns, use the following SQL syntax:

MySQL / SQL Server / Oracle / MS Access:

CREATE TABLE Orders (
    OrderID int NOT NULL,
    OrderNumber int NOT NULL,
    PersonID int,
    PRIMARY KEY (OrderID),
    CONSTRAINT FK_PersonOrder FOREIGN KEY (PersonID)
    REFERENCES Persons(PersonID)
);


--postgre:

create table public.bill8 (id int, moje VARCHAR(1), CONSTRAINT bill8_primary_key PRIMARY KEY (id), CONSTRAINT bill8_foreign_key FOREIGN KEY (moje) REFERENCES public.bill7(moje));
--pozor constraint musí být unikátní napříč db, primary_key by tady nešlo protože už je v bill7 - ERROR:  relation "primary_key" already exists ;

SQL FOREIGN KEY on ALTER TABLE
To create a FOREIGN KEY constraint on the "PersonID" column when the "Orders" table is already created, use the following SQL:

MySQL / SQL Server / Oracle / MS Access:

ALTER TABLE Orders
ADD FOREIGN KEY (PersonID) REFERENCES Persons(PersonID);
To allow naming of a FOREIGN KEY constraint, and for defining a FOREIGN KEY constraint on multiple columns, use the following SQL syntax:

MySQL / SQL Server / Oracle / MS Access:

ALTER TABLE Orders
ADD CONSTRAINT FK_PersonOrder
FOREIGN KEY (PersonID) REFERENCES Persons(PersonID);
DROP a FOREIGN KEY Constraint
To drop a FOREIGN KEY constraint, use the following SQL:

MySQL:

ALTER TABLE Orders
DROP FOREIGN KEY FK_PersonOrder;
SQL Server / Oracle / MS Access:

ALTER TABLE Orders
DROP CONSTRAINT FK_PersonOrder;


--postgre:

ALTER TABLE public.bill8 ADD CONSTRAINT bill8_foreign_key FOREIGN KEY (moje) REFERENCES public.bill7(moje);

ALTER TABLE public.bill8 DROP CONSTRAINT bill8_foreign_key;

In [101]:
cur.execute('''DROP TABLE testDB.ORDERS;''')

0

In [102]:
# The following SQL creates a PRIMARY KEY on the "ID" column when the "Persons" table is created:
cur.execute('''CREATE TABLE testDB.Orders (
    OrderID int NOT NULL,
    OrderNumber int NOT NULL,
    PersonID int,
    PRIMARY KEY (OrderID)
);''')

0

In [103]:
# In MySQL, nullability is a part of the datatype, not a constraint. So:
cur.execute('''ALTER TABLE testDB.Orders ADD FOREIGN KEY (PersonID) REFERENCES Persons(ID);''')

0

In [104]:
# Return data from the Customers table:
cur.execute('''DESCRIBE testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Field,Type,Null,Key,Default,Extra
0,OrderID,int(11),NO,PRI,None,
1,OrderNumber,int(11),NO,,None,
2,PersonID,int(11),YES,MUL,None,


In [105]:
cur.execute('''INSERT INTO testDB.Orders VALUES (1, 1, 5);''')

IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`testdb`.`orders`, CONSTRAINT `orders_ibfk_1` FOREIGN KEY (`PersonID`) REFERENCES `Persons` (`ID`))')

In [106]:
cur.execute('''INSERT INTO testDB.Orders VALUES (1, 1, 1);''')

1

In [107]:
# Return data from the Customers table:
cur.execute('''SELECT * FROM testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,OrderID,OrderNumber,PersonID
0,1,1,1


In [108]:
cur.execute('''ALTER TABLE testDB.Orders DROP FOREIGN KEY(PersonID);''')

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '(PersonID)' at line 1")

In [109]:
cur.execute('''ALTER TABLE testDB.Orders ADD CONSTRAINT FK_PersonOrder FOREIGN KEY (PersonID) REFERENCES Persons(ID);''')

1

In [110]:
# Return data from the Customers table:
cur.execute('''DESCRIBE testDB.ORDERS;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Field,Type,Null,Key,Default,Extra
0,OrderID,int(11),NO,PRI,None,
1,OrderNumber,int(11),NO,,None,
2,PersonID,int(11),YES,MUL,None,


In [111]:
cur.execute('''ALTER TABLE testDB.Orders DROP FOREIGN KEY FK_PersonOrder;''')

0

In [112]:
# Return data from the Customers table:
cur.execute('''DESCRIBE testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Field,Type,Null,Key,Default,Extra
0,OrderID,int(11),NO,PRI,None,
1,OrderNumber,int(11),NO,,None,
2,PersonID,int(11),YES,MUL,None,


In [113]:
pd.options.display.max_colwidth = 999
# Return data from the Customers table:
cur.execute('''SHOW CREATE TABLE testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Table,Create Table
0,Orders,"CREATE TABLE `Orders` (\n `OrderID` int(11) NOT NULL,\n `OrderNumber` int(11) NOT NULL,\n `PersonID` int(11) DEFAULT NULL,\n PRIMARY KEY (`OrderID`),\n KEY `FK_PersonOrder` (`PersonID`),\n CONSTRAINT `orders_ibfk_1` FOREIGN KEY (`PersonID`) REFERENCES `Persons` (`ID`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8"


In [114]:
cur.execute('''ALTER TABLE testDB.Orders DROP FOREIGN KEY orders_ibfk_1 ;''')

0

In [115]:
#Už to jde!!!
cur.execute('''INSERT INTO testDB.Orders VALUES (2, 2, 5); ''')

1

In [116]:
# Return data from the CustomersNew table:
cur.execute('''SELECT * FROM testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,OrderID,OrderNumber,PersonID
0,1,1,1
1,2,2,5


In [117]:
# Return data from the Customers table:
cur.execute('''DESCRIBE testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Field,Type,Null,Key,Default,Extra
0,OrderID,int(11),NO,PRI,None,
1,OrderNumber,int(11),NO,,None,
2,PersonID,int(11),YES,MUL,None,


In [118]:
pd.options.display.max_colwidth = 999
# Return data from the Customers table:
cur.execute('''SHOW CREATE TABLE testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Table,Create Table
0,Orders,"CREATE TABLE `Orders` (\n `OrderID` int(11) NOT NULL,\n `OrderNumber` int(11) NOT NULL,\n `PersonID` int(11) DEFAULT NULL,\n PRIMARY KEY (`OrderID`),\n KEY `FK_PersonOrder` (`PersonID`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8"


In [119]:
#zbyl tam nějaký index???
cur.execute('''ALTER TABLE testdb.`Orders` DROP INDEX `FK_PersonOrder`;''')

0

In [120]:
pd.options.display.max_colwidth = 999
# Return data from the Customers table:
cur.execute('''SHOW CREATE TABLE testDB.Orders;''')
df = pd.DataFrame(tuple(t) for t in cur.fetchall())
columns = [i[0] for i in cur.description]
if len(df) == 0: df = pd.DataFrame([["" for i in range(len(columns))]])
df.columns = columns
df

,Table,Create Table
0,Orders,"CREATE TABLE `Orders` (\n `OrderID` int(11) NOT NULL,\n `OrderNumber` int(11) NOT NULL,\n `PersonID` int(11) DEFAULT NULL,\n PRIMARY KEY (`OrderID`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8"
